In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer
from yellowbrick.features import PCA as YBPCA
from kneed import KneeLocator
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
#import plotly.express as px

In [4]:
bike_winter = pd.read_csv("bike_winter.csv")
bike_winter.head()

,Unnamed: 0,datetime,date,hour,season,holiday,open,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,1,2017-12-01 00:00:00,2017-12-01,0,Winter,No Holiday,Yes,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,2,2017-12-01 01:00:00,2017-12-01,1,Winter,No Holiday,Yes,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,3,2017-12-01 02:00:00,2017-12-01,2,Winter,No Holiday,Yes,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,4,2017-12-01 03:00:00,2017-12-01,3,Winter,No Holiday,Yes,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,5,2017-12-01 04:00:00,2017-12-01,4,Winter,No Holiday,Yes,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0


In [5]:
bike_winter_col = list(bike_winter.columns)
bike_winter_numeric_col = list(bike_winter.describe().columns)
diff_names = []
for i in bike_winter_col:
	if i not in bike_winter_numeric_col:
		diff_names.append(i)
diff_names

['datetime', 'date', 'season', 'holiday', 'open']

In [22]:
bike_winter2 = bike_winter.copy()
bike_winter2.drop(columns = diff_names,inplace=True)
#bike_winter2.drop(columns = ['rent_count'], inplace = True)
bike_winter2.drop(columns = ['rent_count', 'dewpoint_temp'], inplace = True)
bike_winter2.head()

,Unnamed: 0,hour,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall
0,1,0,-5.2,37,2.2,20000,0.0,0.0,0
1,2,1,-5.5,38,0.8,20000,0.0,0.0,0
2,3,2,-6.0,39,1.0,20000,0.0,0.0,0
3,4,3,-6.2,40,0.9,20000,0.0,0.0,0
4,5,4,-6.0,36,2.3,20000,0.0,0.0,0


In [23]:
bike_winter3 = bike_winter2.iloc[: , 2:]
bike_winter3.head()

,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall
0,-5.2,37,2.2,20000,0.0,0.0,0
1,-5.5,38,0.8,20000,0.0,0.0,0
2,-6.0,39,1.0,20000,0.0,0.0,0
3,-6.2,40,0.9,20000,0.0,0.0,0
4,-6.0,36,2.3,20000,0.0,0.0,0


In [24]:
bike_winter3.describe()

,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall
count,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000
mean,-2.540463,49.744907,1.922685,14459.870370,0.298181,0.032824,2.475000
std,5.469763,19.303054,1.156028,6348.564457,0.523243,0.422526,6.983692
min,-17.800000,14.000000,0.000000,540.000000,0.000000,0.000000,0.000000
25%,-6.200000,34.000000,1.000000,9537.500000,0.000000,0.000000,0.000000
50%,-2.200000,47.000000,1.700000,17620.000000,0.000000,0.000000,0.000000
75%,1.500000,63.000000,2.700000,19980.000000,0.452500,0.000000,0.000000
max,10.300000,97.000000,6.700000,20000.000000,2.520000,9.500000,51.000000


In [25]:
_ , p_value = calculate_bartlett_sphericity(bike_winter3)
p_value

0.0

In [26]:
_, kmo_score = calculate_kmo(bike_winter3)
kmo_score 

0.5735963077622983